In [1]:
# Imports
import numpy as np
import torch
import matplotlib.pyplot as plt

from cnp.data import EEGGenerator

from cnp.cnp import StandardEEGConvGNP
from cnp.cov import (
    MultiOutputMeanFieldGaussianLayer,
    MultiOutputInnerprodGaussianLayer,
    MultiOutputKvvGaussianLayer
)

from cnp.experiment import WorkingDirectory
from shutil import copyfile


def to_numpy(tensor):
    return tensor.detach().cpu().numpy()

In [2]:
# model_dir = '/scratches/cblgpu07/em626/kernelcnp/kernelcnp/experiments/eeg/results/1000-8-6-3-50-0/'

# copyfile(f'{model_dir}/convGNP/meanfield/hetero/model_best.pth.tar', 'models/convgnp-meanfield')
# copyfile(f'{model_dir}/convGNP/innerprod/hetero/model_best.pth.tar', 'models/convgnp-innerprod')
# copyfile(f'{model_dir}/convGNP/kvv/hetero/model_best.pth.tar', 'models/convgnp-kvv')

In [3]:
args_num_basis_dim = 64
args_noise_type = 'hetero'
args_jitter = 1e-4
args_num_outputs = 6

device = torch.device('cuda:1')

working_directory = WorkingDirectory('/scratches/cblgpu07/em626/kernelcnp/kernelcnp/notebooks/eeg-results/models')

Root: /scratches/cblgpu07/em626/kernelcnp/kernelcnp/notebooks/eeg-results/models


In [4]:
def load_model(model_name, device, working_directory):
    
    if 'convgnp' in model_name:
        
        output_layer = None

        if 'meanfield' in model_name:
            output_layer = MultiOutputMeanFieldGaussianLayer(num_outputs=6)

        elif 'innerprod' in model_name:
            output_layer = MultiOutputInnerprodGaussianLayer(num_outputs=args_num_outputs,
                                                             num_embedding=args_num_basis_dim,
                                                             noise_type=args_noise_type,
                                                             jitter=args_jitter)

        elif 'kvv' in model_name:
            output_layer = MultiOutputKvvGaussianLayer(num_outputs=args_num_outputs,
                                                       num_embedding=args_num_basis_dim,
                                                       noise_type=args_noise_type,
                                                       jitter=args_jitter)

        model = StandardEEGConvGNP(num_channels=args_num_outputs,
                                   output_layer=output_layer)
        
    elif 'convnp' in model_name:
        model = StandardPredPreyConvNP(input_dim=1, num_samples=16)

    # Load model to appropriate device
    model = model.to(device)

    # Load model from saved state
    load_dict = torch.load(working_directory.file(f'{model_name}', exists=True), map_location=torch.device('cpu'))
    model.load_state_dict(load_dict['state_dict'])
    
    return model

In [5]:
# Load data
train_data = EEGGenerator(split='train',
                          num_total_channels=6,
                          num_target_channels=3,
                          batch_size=1,
                          batches_per_epoch=1,
                          target_length=50)

             F3     F4      F5      F6     FZ     F1     F2
time                                                       
0.000000  3.927  2.421   7.284   4.486  1.383  1.811  1.902
0.003906  4.415  4.862   7.284   5.951  2.848  2.299  3.367
0.007812  4.903  6.327   8.260   7.904  3.825  3.276  4.344
0.011719  6.368  6.327   9.237   9.369  4.313  4.252  4.832
0.015625  7.345  5.351  10.213  11.322  4.313  4.740  4.344
...         ...    ...     ...     ...    ...    ...    ...
0.980469 -0.468 -0.997   6.795 -11.627 -5.941 -3.560 -8.352
0.984375  0.997 -0.509   8.260 -10.651 -4.476 -2.096 -7.375
0.988281  2.462  1.444  10.213  -6.744 -3.988 -0.631 -5.422
0.992188  2.950  4.862  12.166  -1.373 -2.523 -0.142 -3.469
0.996094  3.438  7.304  13.631   2.533 -2.035  0.346 -2.981

[256 rows x 7 columns]


Exception: 

In [ ]:
model_infos = [
    ('ConvGNP (meanfield)', StandardEEGConvGNP, MultiOutputMeanFieldGaussianLayer, 'convgnp-meanfield'),
    ('ConvGNP (linear)', StandardEEGConvGNP, MultiOutputInnerprodGaussianLayer, 'convgnp-innerprod'),
    ('ConvGNP (kvv)', StandardEEGConvGNP, MultiOutputKvvGaussianLayer, 'convgnp-kvv')
]

for model_info in model_infos[:3]:
    
    model = load_model(model_name=model_info[-1],
                       device=device,
                       working_directory=working_directory)
    
    data = list(train_data)[0]
    ones = torch.ones_like(data['m_target']).to(device)
    
    samples, samples_masked = model.sample(x_context=data['x_context'].to(device),
                                           y_context=data['y_context'].to(device),
                                           m_context=data['m_context'].to(device),
                                           x_target=data['x_target'].to(device),
                                           m_target=data['m_target'].to(device),
                                           num_samples=100,
                                           noiseless=False,
                                           double=True)
    
    samples = to_numpy(samples)
    samples_masked = to_numpy(samples_masked)
    
    x_ctx = to_numpy(data['x_context'])
    y_ctx = to_numpy(data['y_context'])
    x_trg = to_numpy(data['x_target'])
    y_trg = to_numpy(data['y_target'])
    
    print(x_trg.shape, samples_masked.shape)
    
    for i in range(6):
        if torch.any(data['m_target'][0, i, :] == 1, dim=0):
            plt.plot(x_ctx[0, :, 0], y_ctx[0, i, :])
            plt.plot(x_trg[0, :, 0], np.mean(samples[:, 0, i, :].T, axis=-1), color='k', alpha=1)
            plt.plot(x_trg[0, :, 0], y_trg[0, i, :])
            plt.show()
        else:
            continue

In [ ]:
x_ctx[0, :, 0].shape, y_ctx[0, :, 0].shape

In [ ]:
y_ctx[0, :, :].shape